In [ ]:
def threshold_alpha_beta(threshold):
    successes = ee.Image.constant(0).rename('alpha')
    failures = ee.Image.constant(0).rename('beta')
    for hist_year in range(HIST_START, HIST_END + 1):
        ehe_histyear = ehe(era5.filterDate(str(hist_year) + '-01-01', str(hist_year) + '-12-31'))
        successes = successes.add(ehe_histyear.gte(threshold).multiply(ehe_histyear.lt(threshold + 5))).add(1)
        failures = failures.add(ehe_histyear.lt(threshold).add(ehe_histyear.gte(threshold + 5)).min(1)).add(1)
    alpha = successes.set('source', 'era5').set('threshold', threshold)
    beta = failures.set('source', 'era5').set('threshold', threshold)
    ic = ee.ImageCollection([alpha, beta])
    twoband_image = ic.toBands().rename(['alpha', 'beta'])
return twoband_image

In [ ]:
def parameter_prime(param, target_year, threshold, flip):
    cnp = current_naive_probs.select('threshold_' + str(threshold))
    fnp = future_naive_probs.select('threshold_' + str(threshold))
    if flip:
        return param.add((((fnp.multiply(-1).add(1)).subtract(cnp.multiply(-1).add(1))).divide(2050 - 2020)).multiply(target_year - 2020)).multiply(0.1)
    else:
        return param.add(((fnp.subtract(cnp)).divide(2050 - 2020)).multiply(target_year - 2020)).multiply(0.1)